# Libraries

In [8]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
# Load data using preprocessing.py functions
import sys
import os
sys.path.append(os.path.abspath(".."))
from src.preprocessing import load_data, split_data, get_preprocessor